In [38]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET
import numpy as np
import re


In [39]:
DATE_TEMPLATE = re.compile(r'\d{2}\.\d{2}\.\d{4}')

In [40]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(root=root):
    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s")

        remove_explicit_from_sentence(source_sentence)

        text = ''.join(source_sentence.itertext())
        dict_of_leads[id] = text;
    return dict_of_leads

def remove_explicit_from_sentence(xml_sentence):
    def remove_target_node(xml_sentence, target_template):
        for node in xml_sentence:
            if re.search(target_template, node.attrib["lemma"]):
                xml_sentence.remove(node)
    # to remove
    phoneno = re.compile(r'\d{11}')
    date = DATE_TEMPLATE

    remove_target_node(xml_sentence, phoneno)
    remove_target_node(xml_sentence, date)
      
    

one = make_dict_of_leads()['0']


"""
   <b>
      <s>
         21.06.2016
         <P n0="" lemma="89140580517">89140580517</P>
         хочет
         <W n1="" lemma="buy">купить</W>
         <W n2="" lemma="ноутбук">ноутбук</W>
         за 17000 руб ездить
         <P n3="" lemma="Якутия">Якутия</P>
         .
      </s>
   </b>
"""


one

'самсунг с4 купить.'

In [41]:
# todo: помнить о тексте лида. Там выделены факты прямо в разметке - полезно при выводе информации в веб-интерфейсе

def compare_facts_to_leads(root=root):
    facts_grouped_by_lead = dict()

    for i in root.find("document").find('facts'):
        lead_id = i.attrib['LeadID']
        if facts_grouped_by_lead.get(lead_id):
            facts_grouped_by_lead[lead_id].append(i)
        else:
            facts_grouped_by_lead[lead_id] = [i]
    return facts_grouped_by_lead

def make_common_table():
    calls = df()
    
    facts = compare_facts_to_leads()
    leads = make_dict_of_leads()

    for lead in facts:
        try:
            elems = facts[lead]
            one_sentence = leads[lead]
            cols = ["conversation"]
            values = [one_sentence]
            for fact_name in elems:
                for fact_field in fact_name:
                    cols.append(fact_name.tag + "_" + fact_field.tag)
                    values.append(fact_field.attrib["val"])
            one_row = pd.DataFrame([values], columns=cols)
            calls = calls.append(one_row)

            values = []
            cols = []
        except ValueError as e:
            print(e)
            print(lead, leads[lead])
    return calls


In [42]:
calls = make_common_table()

calls.to_excel("whole_table.xlsx")
calls

,CallDate_Date,CustomerBuys_Word,CustomerPhone_Phone,CustomerPlace_Place,CustomerSells_Word,Matter_Notebook,Matter_Other,Matter_Phone,Matter_TV,Matter_Tablet,Pawn_Word,Repare_Word,conversation
0,24.06.2016,КУПЛЯ,89644232158,NaN,NaN,NaN,NaN,NaN,NaN,НОТ САМСУНГ,NaN,NaN,нот самсунг купить.
0,25.06.2016,КУПЛЯ,89656765854,NaN,NaN,NaN,NaN,NaN,NaN,ПЛАНШЕТ,NaN,NaN,планшет купить в офис.
0,25.06.2016,NaN,89000000002,NaN,ПРОДАЖА,NaN,ШУРУПОВЕРТ МАКИТА,NaN,NaN,NaN,NaN,NaN,шуруповерт макита продать.
0,25.06.2016,КУПЛЯ,89142623707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,куп.
0,24.06.2016,NaN,89248742895,NaN,NaN,NaN,NaN,АЙФОН,NaN,NaN,NaN,NaN,айфон.
0,25.06.2016,NaN,89000000003,NaN,ПРОДАЖА,NaN,СТИРАЛЬНАЯ МАШИНА,NaN,NaN,NaN,NaN,NaN,стиральная машина продать.
0,25.06.2016,КУПЛЯ,89243675693,NaN,NaN,NaN,NaN,NaN,NaN,ПЛАНШЕТ,NaN,NaN,планшет куп.
0,25.06.2016,NaN,89000000004,NaN,NaN,NaN,БЕЛЫЕ СЛИВОЧКИ,NaN,NaN,NaN,В ЛОМБАРД,NaN,белые сливочки в ломбард.
0,25.06.2016,КУПЛЯ,89248738577,NaN,NaN,NaN,NaN,NaN,NaN,ПЛАНШЕТ НОТ,NaN,NaN,планшет нот куп.
0,25.06.2016,NaN,89627362387,NaN,ПРОДАЖА,NaN,NaN,АЙПОТ,NaN,NaN,NaN,NaN,айпот продать.


In [43]:
#pawn = calls[calls.Pawn_Word.notnull()].to_excel("pawn.xlsx")
buy_out = calls[calls.CustomerBuys_Word.notnull()].to_excel("buy_out.xlsx")
calls[calls.CustomerSells_Word.notnull()].to_excel("sell.xlsx")
repare = calls[calls.Repare_Word.notnull()].to_excel("repare.xlsx")
